In [11]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

# Define the required scopes for Google Docs and Google Drive
SCOPES = ['https://www.googleapis.com/auth/documents.readonly', 'https://www.googleapis.com/auth/drive.readonly']

def authenticate_google_docs():
    # Specify the path to 'credentials.json'
    creds_path = 'credentials.json'
    
    # Perform OAuth2 authentication and generate 'token.json'
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
    creds = flow.run_local_server(port=0)
    
    # Save the credentials to 'token.json' for future use
    with open('token.json', 'w') as token_file:
        token_file.write(creds.to_json())
    
    print("Authentication complete. Token saved to 'token.json'.")

authenticate_google_docs()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=442262473103-8bfbbiqctfglprh6pmtuvubmk0uir78h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58866%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=LNnA0HrrxBki0KLczMI11piQlAKG34&access_type=offline
Authentication complete. Token saved to 'token.json'.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def retrieve_and_print_table(url):
    # Initialize a variable that will store each column from html table
    X =[]
    character=[]
    Y = []
    # Sending a GET request to the Google Doc URL
    response = requests.get(url)
    
    if response.status_code == 200: # A response code that shows success
       
        # Parsed the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # finding the table in the body of the document
        table = soup.find('table')

        # Iterate through the rows of the table, and start at index 1 because index 0 have cells as column name which will affect the datatypes if included 
        for row in table.find_all('tr')[1:]:
            CELLS = row.find_all('td')
            
            # i ensured cells is enough for our variable defined above. More like quality control check
            if len(CELLS) >= 3:
                X.append(CELLS[0].text.strip())
                character.append(CELLS[1].text.strip())
                Y.append(CELLS[2].text.strip())
            else:
                print(f'Cell number : {len(CELLS)} not enough for our variables X, character and Y')
    else:
        print(f"Failed to retrieve the document. Status code: {response.status_code}")

# Created a dataframe for our scraped table for further processing to achieve ur task
    
    content = {'X': X, 'Character': character, 'Y':Y}
    if content:
        content_df = pd.DataFrame(content)
        print(content_df)

# DATA WRANGLING PHASE
    # Convert the X and Y columns to integers if they are strings
    # This will make sure the values are properly cast before arithmetic operations
    
    content_df['X'] = pd.to_numeric(content_df['X'], errors='coerce')
    content_df['Y'] = pd.to_numeric(content_df['Y'], errors='coerce')

    # Ensure there are no NaN values after conversion
    if content_df[['X', 'Y']].isnull().any().any():
        print("\nThere are missing values in X or Y columns.")

# DRAFTING THE GRID THAT DISPLAY THE PATTERN OF CHARACTER COLUMN 
    else:
        # Determine the size of the grid based on the max x and y coordinates
        max_x = content_df['X'].max() + 1
        max_y = content_df['Y'].max() + 1
        print(f"\nMax X: {max_x}, Max Y: {max_y}")

        # Create an empty grid with spaces (' ') as placeholders. More building the foundation of our structure which the pattern of characters will rest on
        
        grid = [[' ' for _ in range(max_x)] for _ in range(max_y)]

        # Fill the grid with the characters from content_df
        for index, row in content_df.iterrows():
            x = row['X']
            y = row['Y']
            character = row['Character']
            
            # Print debug information for each row to ensure values are correct placing character on the grid
            print(f"Placing character '{character}' at (X: {x}, Y: {y})")
            
            grid[y][x] = character  # Place the character at the appropriate position in the grid

        print("\nGenerated grid:")
        for row in grid:
            print("".join(row)) # this print function is to eliminate any space between our characters in the pattern for a close fit.

# Google Doc URL- calling the function
url = "https://docs.google.com/document/d/e/2PACX-1vSHesOf9hv2sPOntssYrEdubmMQm8lwjfwv6NPjjmIRYs_FOYXtqrYgjh85jBUebK9swPXh_a5TJ5Kl/pub"
retrieve_and_print_table(url)

In [65]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def retrieve_and_print_table(url):
    # Initialize a variable that will store each column from html table
    X =[]
    character=[]
    Y = []
    # Send a GET request to the public Google Doc URL
    response = requests.get(url)
    
    if response.status_code == 200:
       
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # finding the table in the body of the document
        table = soup.find('table')

        # Iterate through the rows of the table
        for row in table.find_all('tr')[1:]:
            CELLS = row.find_all('td')
            
            # i ensured cells is enough for our variable defined above
            if len(CELLS) >= 3:
                X.append(CELLS[0].text.strip())
                character.append(CELLS[1].text.strip())
                Y.append(CELLS[2].text.strip())
            else:
                print(f'Cell number : {len(CELLS)} not enough for our variables X, character and Y')
    else:
        print(f"Failed to retrieve the document. Status code: {response.status_code}")

# Create a dataframe for our data
    content = {'X': X, 'Character': character, 'Y':Y}
    if content:
        content_df = pd.DataFrame(content)
        print(content_df)
        
    # Convert the X and Y columns to integers if they are strings
    # This will make sure the values are properly cast before arithmetic operations
    content_df['X'] = pd.to_numeric(content_df['X'], errors='coerce')
    content_df['Y'] = pd.to_numeric(content_df['Y'], errors='coerce')

    # Ensure there are no NaN values after conversion
    if content_df[['X', 'Y']].isnull().any().any():
        print("\nThere are missing values in X or Y columns after conversion.")
    else:
        # Determine the size of the grid based on the max x and y coordinates
        max_x = content_df['X'].max() + 1
        max_y = content_df['Y'].max() + 1
        print(f"\nMax X: {max_x}, Max Y: {max_y}")

        # Create an empty grid with spaces (' ') as placeholders
        grid = [[' ' for _ in range(max_x)] for _ in range(max_y)]

        # Fill the grid with the characters from content_df
        for index, row in content_df.iterrows():
            x = row['X']
            y = row['Y']
            character = row['Character']
            
            # Print debug information for each row to ensure values are correct
            print(f"Placing character '{character}' at (X: {x}, Y: {y})")
            
            grid[y][x] = character  # Place the character at the appropriate position in the grid

        # Print the grid
        print("\nGenerated grid:")
        for row in grid:
            print("".join(row))


# Your public Google Doc URL
url = "https://docs.google.com/document/d/e/2PACX-1vSHesOf9hv2sPOntssYrEdubmMQm8lwjfwv6NPjjmIRYs_FOYXtqrYgjh85jBUebK9swPXh_a5TJ5Kl/pub"
retrieve_and_print_table(url)


      X Character  Y
0    87         █  3
1    23         ░  2
2    61         █  4
3     2         ░  1
4    65         █  4
..   ..       ... ..
327  21         █  1
328  71         ░  2
329  69         █  2
330   1         █  2
331  27         █  0

[332 rows x 3 columns]

Max X: 90, Max Y: 7
Placing character '█' at (X: 87, Y: 3)
Placing character '░' at (X: 23, Y: 2)
Placing character '█' at (X: 61, Y: 4)
Placing character '░' at (X: 2, Y: 1)
Placing character '█' at (X: 65, Y: 4)
Placing character '░' at (X: 31, Y: 5)
Placing character '░' at (X: 30, Y: 6)
Placing character '█' at (X: 20, Y: 4)
Placing character '█' at (X: 35, Y: 1)
Placing character '█' at (X: 2, Y: 6)
Placing character '█' at (X: 52, Y: 1)
Placing character '█' at (X: 12, Y: 0)
Placing character '█' at (X: 66, Y: 6)
Placing character '█' at (X: 69, Y: 4)
Placing character '█' at (X: 85, Y: 0)
Placing character '█' at (X: 55, Y: 6)
Placing character '█' at (X: 77, Y: 3)
Placing character '█' at (X: 9, Y: 0)
Plac

In [8]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

def retrieve_and_print_table(url):
    # Authenticate using OAuth2 credentials (Assuming token.json already exists)
    SCOPES = ['https://www.googleapis.com/auth/documents.readonly']
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # Create a Docs API service instance
    service = build('docs', 'v1', credentials=creds)
    
    # Extract the document ID from the URL
    document_id = url.split("/d/")[1].split("/")[0] 
    
    # Retrieve the document
    document = service.documents().get(documentId=document_id).execute()
    print(document)
            
url="https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub"
retrieve_and_print_table(url)


HttpError: <HttpError 404 when requesting https://docs.googleapis.com/v1/documents/e?alt=json returned "Requested entity was not found.". Details: "Requested entity was not found.">